In [1]:
import time
import pandas as pd
import numpy as np
from finstore.finstore import Finstore
from finstore.finstore_s3 import PartitionType

In [ ]:
f = Finstore(market_name="crypto_binance", timeframe="1m", partition_type = PartitionType.DAILY)

# Generate Mock Data

In [3]:
num_days = 10
total_rows = num_days * 1440
all_dates = pd.date_range("2025-01-01", periods=total_rows, freq='min', tz='UTC')
print(f"Generated {len(all_dates)} timestamps.")

# Create random OHLCV data with a timestamp column
df = pd.DataFrame({
     'timestamp': all_dates,
     'open': np.random.rand(len(all_dates)),
     'high': np.random.rand(len(all_dates)),
     'low': np.random.rand(len(all_dates)),
     'close': np.random.rand(len(all_dates)),
     'volume': np.random.randint(100, 1000, size=len(all_dates))
})

Generated 14400 timestamps.


In [4]:
df

,timestamp,open,high,low,close,volume
0,2025-01-01 00:00:00+00:00,0.038043,0.654339,0.419680,0.980483,487
1,2025-01-01 00:01:00+00:00,0.060033,0.484785,0.296628,0.973226,318
2,2025-01-01 00:02:00+00:00,0.125746,0.935153,0.690261,0.629535,157
3,2025-01-01 00:03:00+00:00,0.000360,0.881578,0.238030,0.653138,674
4,2025-01-01 00:04:00+00:00,0.169392,0.366717,0.166004,0.966910,586
...,...,...,...,...,...,...
14395,2025-01-10 23:55:00+00:00,0.683137,0.159707,0.762966,0.349553,280
14396,2025-01-10 23:56:00+00:00,0.131895,0.608151,0.974880,0.524510,521
14397,2025-01-10 23:57:00+00:00,0.793416,0.030099,0.282857,0.867185,131
14398,2025-01-10 23:58:00+00:00,0.155255,0.194681,0.568610,0.566962,591


# Store Data with Daily Partitioning

In [19]:
%%time
f.s3.store(symbol="BTC/USDT", df=df)

Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-01/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-02/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-03/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-04/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-05/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-06/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-07/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-08/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-09/ohlcv_data.parquet
Stored 1440 rows to s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-10/ohlcv_data.parquet
CPU times: user 142 

# Fetching Data

In [20]:
%%time
fetched_df = f.s3.fetch(symbol="BTC/USDT", 
                        start_date="2025-01-01", 
                        end_date="2025-01-10")

CPU times: user 106 ms, sys: 1.78 ms, total: 108 ms
Wall time: 73.7 ms


In [21]:
%%time
fetched_df = f.s3.fetch(symbol="BTC/USDT", 
                        start_date="2025-01-01", 
                        end_date="2025-01-5")

CPU times: user 55.1 ms, sys: 3.97 ms, total: 59 ms
Wall time: 39.9 ms


In [22]:
%%time
fetched_df = f.s3.fetch(symbol="BTC/USDT")

CPU times: user 48.2 ms, sys: 48.1 ms, total: 96.4 ms
Wall time: 79.9 ms


# Deleting Data

In [23]:
%%time
f.s3.delete(symbol="BTC/USDT", partition_value="2025-01-05")

Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-05/ohlcv_data.parquet
CPU times: user 6.39 ms, sys: 0 ns, total: 6.39 ms
Wall time: 6.96 ms


In [24]:
f.s3.fetch(symbol="BTC/USDT")

,timestamp,open,high,low,close,volume,partition
0,2025-01-01 00:00:00+00:00,0.038043,0.654339,0.419680,0.980483,487,2025-01-01
1,2025-01-01 00:01:00+00:00,0.060033,0.484785,0.296628,0.973226,318,2025-01-01
2,2025-01-01 00:02:00+00:00,0.125746,0.935153,0.690261,0.629535,157,2025-01-01
3,2025-01-01 00:03:00+00:00,0.000360,0.881578,0.238030,0.653138,674,2025-01-01
4,2025-01-01 00:04:00+00:00,0.169392,0.366717,0.166004,0.966910,586,2025-01-01
...,...,...,...,...,...,...,...
12955,2025-01-10 23:55:00+00:00,0.683137,0.159707,0.762966,0.349553,280,2025-01-10
12956,2025-01-10 23:56:00+00:00,0.131895,0.608151,0.974880,0.524510,521,2025-01-10
12957,2025-01-10 23:57:00+00:00,0.793416,0.030099,0.282857,0.867185,131,2025-01-10
12958,2025-01-10 23:58:00+00:00,0.155255,0.194681,0.568610,0.566962,591,2025-01-10


In [25]:
%%time
f.s3.delete(symbol="BTC/USDT")

Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-01/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-02/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-03/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-04/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-06/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-07/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-08/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-09/ohlcv_data.parquet
Deleted s3://my-ohlcv-bucket/crypto_binance/1m/BTC/USDT/2025-01-10/ohlcv_data.parquet
CPU times: user 31.7 ms, sys: 0 ns, total: 31.7 ms
Wall time: 55.7 ms


In [26]:
f.s3.fetch(symbol="BTC/USDT")

""
